In [12]:
#Imports
import pandas as pd
import numpy as np

In [13]:
# Load CNN scores
scores_run1 = pd.read_csv('/home/hep/an1522/dark_tridents_wspace/outputs/inference/run1_samples/run1_NuMI_nu_overlay_larcv_cropped_scores.csv')
scores_run3 = pd.read_csv('/home/hep/an1522/dark_tridents_wspace/outputs/inference/run3_samples/run3_nu_overlay_larcv_cropped_scores.csv')

print(f"Run1 CNN scores: {len(scores_run1)} events")
print(f"Run3 CNN scores: {len(scores_run3)} events")

Run1 CNN scores: 13770 events
Run3 CNN scores: 11526 events


In [14]:
# Load Luis's pickle files with MC weights
pkl_run1 = pd.read_pickle('/home/hep/an1522/dark_tridents_wspace/pyhf_files/Analysis_datasets/run1_nu_overlay_CNN.pkl')
pkl_run3 = pd.read_pickle('/home/hep/an1522/dark_tridents_wspace/pyhf_files/Analysis_datasets/run3_nu_overlay_CNN.pkl')

print(f"\nRun1 pickle: {len(pkl_run1)} events")
print(f"Run3 pickle: {len(pkl_run3)} events")
print(f"\nPickle columns: {pkl_run1.columns.tolist()}")


Run1 pickle: 921140 events
Run3 pickle: 11532 events

Pickle columns: ['run_number', 'subrun_number', 'event_number', 'reco_vertex_size', 'reco_asso_showers', 'reco_asso_tracks', 'reco_vertex_x', 'reco_vertex_y', 'reco_vertex_z', 'reco_shower_energy_max', 'reco_track_calo_energy_max', 'reco_flash_time', 'reco_shower_dirx', 'reco_shower_diry', 'reco_shower_dirz', 'ppfx_cv_good', 'spine_tune_good']


In [8]:
# Create unique event ID for matching
scores_run1['rse_id'] = scores_run1['run_number'].astype(str) + "_" + scores_run1['subrun_number'].astype(str) + "_" + scores_run1['event_number'].astype(str)
scores_run3['rse_id'] = scores_run3['run_number'].astype(str) + "_" + scores_run3['subrun_number'].astype(str) + "_" + scores_run3['event_number'].astype(str)

pkl_run1['rse_id'] = pkl_run1['run_number'].astype(int).astype(str) + "_" + pkl_run1['subrun_number'].astype(int).astype(str) + "_" + pkl_run1['event_number'].astype(int).astype(str)
pkl_run3['rse_id'] = pkl_run3['run_number'].astype(int).astype(str) + "_" + pkl_run3['subrun_number'].astype(int).astype(str) + "_" + pkl_run3['event_number'].astype(int).astype(str)

In [9]:
# Merge CNN scores with pickle files
merged_run1 = pd.merge(scores_run1, pkl_run1[['rse_id', 'ppfx_cv_good', 'spine_tune_good']], on='rse_id', how='inner')
merged_run3 = pd.merge(scores_run3, pkl_run3[['rse_id', 'ppfx_cv_good', 'spine_tune_good']], on='rse_id', how='inner')

print(f"Merged Run1: {len(merged_run1)} events (matched {100*len(merged_run1)/len(scores_run1):.1f}%)")
print(f"Merged Run3: {len(merged_run3)} events (matched {100*len(merged_run3)/len(scores_run3):.1f}%)")

Merged Run1: 13770 events (matched 100.0%)
Merged Run3: 11526 events (matched 100.0%)


In [10]:
# Calculate MC weight
merged_run1['weight'] = merged_run1['ppfx_cv_good'] * merged_run1['spine_tune_good']
merged_run3['weight'] = merged_run3['ppfx_cv_good'] * merged_run3['spine_tune_good']

# Fix bad weights (from Luis's code)
for df in [merged_run1, merged_run3]:
    df.loc[df['weight'] <= 0, 'weight'] = 1.0
    df.loc[df['weight'] == np.inf, 'weight'] = 1.0
    df.loc[df['weight'] > 50, 'weight'] = 1.0
    df.loc[np.isnan(df['weight']), 'weight'] = 1.0

print(f"\nRun1 weight stats: mean={merged_run1['weight'].mean():.3f}, std={merged_run1['weight'].std():.3f}")
print(f"Run3 weight stats: mean={merged_run3['weight'].mean():.3f}, std={merged_run3['weight'].std():.3f}")


Run1 weight stats: mean=0.965, std=0.709
Run3 weight stats: mean=0.955, std=0.608


In [1]:
# merged_run1.to_csv('/home/hep/an1522/dark_tridents_wspace/outputs/inference/run1_samples/run1_nu_overlay_merged_with_weights.csv', index=False)
# merged_run3.to_csv('/home/hep/an1522/dark_tridents_wspace/outputs/inference/run3_samples/run3_nu_overlay_merged_with_weights.csv', index=False)
# print("\nSaved merged nu_overlay files!")